#### 

In [2]:
import numpy as np
import pandas as pd

In [41]:
X = np.array([[1,0],
              [0,1],
              [1,-1],
              [1,1]])
Y = np.array([3,3,1,7])
Xt = X.T
#x = np.matrix([[1,2,3], [4,5,6]])

Xt @ X
np.linalg.inv(Xt@X)@Xt@Y

array([3.66666667, 3.        ])